In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions
from IPython.display import display, Math, Latex
import numpy_financial as npf
import matplotlib.pyplot as plt
import random
from datetime import datetime

To change: 
- convert exchange rates at the beginning
- take monthly values instead
- flip x and y column headers
- implement transaction cost considerations

In [26]:
start_date = "2020-10-01"
end_date = "2024-11-14"

In [22]:
# cleaning function

tickers_file_name = "Tickers_Example.csv"
tickers_file_original = pd.read_csv(tickers_file_name, header = None, names=['Ticker'])
print(tickers_file_original.Ticker.tolist())

total_investment = 1000000
drop_tickers = []

for tick in tickers_file_original['Ticker']:    

    try:

        ticker = yf.Ticker(tick) 

        # check if ticker is available
        if ticker.info.get('quoteType')=='NONE':
            # print("phewww", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        # check if its in US or CAD market
        if ticker.fast_info.get('currency') not in ['USD', 'CAD']:
            # print("hmmm", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]
            continue

        history = ticker.history(start = start_date, end = end_date)

        trading_days = history['Volume'].resample('ME').count()
        volume = history['Volume'].resample('ME').sum()
        trading_days = trading_days[trading_days>=18] # how does this work?
        volume = volume[volume.index==trading_days.index]
        monthly_volume = volume.mean()

        if monthly_volume<100000: 
            # print("sheesh", tick)
            tickers_file_original = tickers_file_original[tickers_file_original['Ticker'] != tick]

    except:
            # print("ufff", tick)
            pass
        
tickers_file_original = tickers_file_original.reset_index(drop=True)

['AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [5]:
print(tickers_file_original.Ticker.tolist())

['AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB']


In [7]:
difference = {'AAPL', 'ABBV', 'ABT', 'ACN', 'AGN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CELG', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MON', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTN', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB'} - {'AAPL', 'ABBV', 'ABT', 'ACN', 'AIG', 'AMZN', 'AXP', 'BA', 'BAC', 'BB.TO', 'BIIB', 'BK', 'BLK', 'BMY', 'C', 'CAT', 'CL', 'KO', 'LLY', 'LMT', 'MO', 'MRK', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RY.TO', 'SHOP.TO', 'T.TO', 'TD.TO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB'}
difference

{'AGN', 'CELG', 'MON', 'RTN'}

In [40]:
# Define stock list and market index
stock_list = tickers_file_original.Ticker.tolist()
market_indices = ["^GSPC", '^GSPTSE']  # S&P 500

# Step 1: Fetch historical data
data = yf.download(stock_list + market_indices, interval = '1mo', start=start_date, end=end_date)["Close"]
data.index = data.index.strftime('%Y-%m-%d')

# Separate stock prices and market index prices
stock_prices = data[stock_list]
SP = data[market_indices[0]]
TSX = data[market_indices[1]]
display(data)
display(exchange_rate)

[*********************100%***********************]  39 of 39 completed


Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB,^GSPC,^GSPTSE
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01,108.860001,85.099998,105.110001,216.910004,31.490000,151.807495,91.239998,144.389999,23.700001,5.980000,...,122.822998,22.780001,58.779999,144.589996,305.140015,177.190002,157.110001,38.950001,3269.959961,15580.599609
2020-11-01,119.050003,104.580002,108.220001,249.089996,38.439999,158.401993,118.589996,210.710007,28.160000,7.650000,...,139.776993,25.049999,69.279999,161.250000,336.339996,204.080002,171.070007,43.209999,3621.629883,17190.300781
2020-12-01,132.690002,107.150002,109.489998,261.209991,37.860001,162.846497,120.910004,214.059998,30.309999,8.440000,...,143.731995,25.209999,71.919998,164.130005,350.679993,208.220001,168.399994,46.590000,3756.070068,17433.400391
2021-01-01,131.960007,102.480003,123.589996,241.919998,37.439999,160.309998,116.260002,194.190002,29.650000,17.959999,...,139.429001,26.389999,72.459999,165.690002,333.579987,197.470001,155.000000,42.849998,3714.239990,17337.000000
2021-02-01,121.260002,107.739998,119.779999,250.899994,43.950001,154.646500,135.259995,212.009995,34.709999,12.870000,...,164.873001,25.500000,77.139999,172.270004,332.220001,205.960007,157.830002,50.000000,3811.149902,18060.300781
2021-03-01,122.150002,108.220001,119.839996,276.250000,46.209999,154.703995,141.440002,254.720001,38.689999,10.540000,...,138.742996,25.030001,81.959999,188.990005,372.070007,220.410004,169.990005,55.310001,3972.889893,18700.699219
2021-04-01,131.460007,111.500000,120.080002,289.970001,48.450001,173.371002,153.350006,234.309998,40.529999,10.870000,...,145.089996,25.500000,84.500000,180.509995,398.799988,222.089996,203.860001,59.349998,4181.169922,19108.300781
2021-05-01,124.610001,113.199997,116.650002,282.160004,52.840000,161.153503,160.130005,247.020004,42.389999,12.400000,...,148.031998,27.100000,86.900002,189.820007,411.920013,224.729996,214.600006,60.779999,4204.109863,19731.000000
2021-06-01,136.960007,112.639999,115.930000,294.790009,47.599998,172.007996,165.229996,239.559998,41.230000,15.150000,...,181.287003,27.799999,86.870003,192.300003,400.440002,219.929993,207.970001,56.970001,4297.500000,20165.599609


,Close
Date,
2020-10-01,1.33153
2020-11-01,1.29826
2020-12-01,1.27507
2021-01-01,1.27760
2021-02-01,1.27387
2021-03-01,1.26208
2021-04-01,1.22762
2021-05-01,1.20778
2021-06-01,1.23962


In [30]:
# convert all to CAD
exchange = 'USDCAD=X'
exchange_ticker = yf.Ticker(exchange)
exchange_rate = exchange_ticker.history(interval = '1mo', start=start_date, end=end_date)["Close"]
exchange_rate = pd.DataFrame(exchange_rate)
exchange_rate.index = exchange_rate.index.strftime('%Y-%m-%d')

for ticker in stock_list:
    ticker_name = yf.Ticker(ticker)
    currency = ticker_name.fast_info["currency"]
    if currency == 'USD':
        data[ticker] = exchange_rate['Close'] * data[ticker]
    elif currency == 'CAD':
        continue
    else:
        raise ValueError(f"Unsupported currency: {currency}")

data['^GSPC'] = data['^GSPC'] * exchange_rate['Close']
data['^GSPTSE'] = data['^GSPTSE'] * exchange_rate['Close']

display(data)

Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB,^GSPC,^GSPTSE
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01,144.950354,113.313199,139.957116,288.822172,41.929879,202.136230,121.488792,192.259612,31.557261,5.980000,...,122.822998,22.780001,58.779999,192.525914,406.303076,235.933800,209.196675,51.863094,4354.049705,20746.035407
2020-11-01,154.557854,135.772030,140.497696,323.383572,49.905112,205.646967,153.960646,273.556368,36.559001,7.650000,...,139.776993,25.049999,69.279999,209.344421,436.656755,264.948898,222.093343,56.097812,4701.817116,22317.479437
2020-12-01,169.189035,136.623747,139.607406,333.061011,48.274149,207.640675,154.168713,272.941471,38.647370,8.440000,...,143.731995,25.209999,71.919998,209.277237,447.141521,265.495067,214.721772,59.405509,4789.252082,22228.804999
2021-01-01,168.592111,130.928457,157.898585,309.077002,47.833344,204.812061,148.533785,248.097157,37.880841,17.959999,...,139.429001,26.389999,72.459999,211.685555,426.181808,252.287683,198.028008,54.745160,4745.313197,22149.752066
2021-02-01,154.469478,137.246750,152.584146,319.613973,55.986587,196.999535,172.303648,270.073170,44.216026,12.870000,...,164.873001,25.500000,77.139999,219.449589,423.205090,262.366272,201.054903,63.693500,4854.909493,23006.475199
2021-03-01,154.163068,136.582294,151.247657,348.649587,58.320714,195.248811,178.508592,321.477007,48.829872,10.540000,...,138.742996,25.030001,81.959999,238.520497,469.582098,278.175047,214.540978,69.805644,5014.104693,23601.777613
2021-04-01,161.382934,136.879631,147.412613,355.972975,59.478190,212.833711,188.255535,287.643641,49.755437,10.870000,...,145.089996,25.500000,84.500000,221.597680,489.574843,272.642123,250.262615,72.859245,5132.887843,23457.732312
2021-05-01,150.501467,136.720693,140.887539,340.787210,63.819096,194.637979,193.401818,298.345822,51.197794,12.400000,...,148.031998,27.100000,86.900002,229.260809,497.508755,271.424395,259.189596,73.408867,5077.639826,23830.707250
2021-06-01,169.778359,139.630793,143.709144,365.427582,59.005909,213.224546,204.822402,296.963357,51.109531,15.150000,...,181.287003,27.799999,86.870003,238.378924,496.393424,272.629611,257.803767,70.621151,5327.266822,24997.679989


In [34]:
# Step 2: Calculate beta and covariance
returns = stock_prices.pct_change().dropna()
SP_returns = SP.pct_change().dropna()
TSX_returns = TSX.pct_change().dropna()

# Calculate betas
SP_betas = {}
TSX_betas = {}
betas_combined = {}
for stock in stock_list:
    cov1 = np.cov(returns[stock], SP_returns)[0][1]
    var1 = np.var(SP_returns)
    SP_betas[stock] = np.round(cov1 / var1, 3)

    cov2 = np.cov(returns[stock], TSX_returns)[0][1]
    var2 = np.var(TSX_returns)
    TSX_betas[stock] = np.round(cov2 / var2, 3)

    betas_combined[stock] = np.round(SP_betas[stock] + TSX_betas[stock], 3)

# Select stocks with beta close to 1
beta_target = 1
selected_stocks = sorted(betas_combined.keys(), key=lambda x: abs(betas_combined[x] - beta_target))[:18]
print("Combined beta values: ")
display(betas_combined)
print("Selected stocks: ")
print(selected_stocks)

Combined beta values: 


{'AAPL': 2.245,
 'ABBV': 1.311,
 'ABT': 1.524,
 'ACN': 2.651,
 'AIG': 2.211,
 'AMZN': 2.351,
 'AXP': 3.1,
 'BA': 3.66,
 'BAC': 3.175,
 'BB.TO': 2.444,
 'BIIB': 0.118,
 'BK': 2.587,
 'BLK': 3.466,
 'BMY': 1.094,
 'C': 3.17,
 'CAT': 3.303,
 'CL': 0.822,
 'KO': 1.162,
 'LLY': 0.642,
 'LMT': 0.742,
 'MO': 1.702,
 'MRK': 0.807,
 'PEP': 1.062,
 'PFE': 1.098,
 'PG': 0.726,
 'PM': 1.428,
 'PYPL': 3.525,
 'QCOM': 2.69,
 'RY.TO': 1.656,
 'SHOP.TO': 5.05,
 'T.TO': 1.353,
 'TD.TO': 1.566,
 'TXN': 2.168,
 'UNH': 1.158,
 'UNP': 2.338,
 'UPS': 2.447,
 'USB': 2.817}

Selected stocks: 
['PEP', 'BMY', 'PFE', 'UNH', 'KO', 'CL', 'MRK', 'LMT', 'PG', 'ABBV', 'T.TO', 'LLY', 'PM', 'ABT', 'TD.TO', 'RY.TO', 'MO', 'BIIB']


In [36]:
# Step 3: Prepare for portfolio optimization
selected_prices = stock_prices[selected_stocks] # takes the stock prices of the selected stocks, DataFrame
#display(selected_prices)
mu = expected_returns.mean_historical_return(selected_prices) # calculates mean historical returns
S = risk_models.sample_cov(selected_prices) # calculates covariance of each pairof stocks - how stock returns move together
#display(S)

# Optimize portfolio using the EfficientFrontier object
ef = EfficientFrontier(mu, S) # to optimize portfolio weights
ef.add_objective(objective_functions.L2_reg, gamma=0.1)  # Add L2 regularization 
    # to prevent any single stock from having excessive large weight, gamma controls strenght of penalty 
    # figure out what L2_reg is exactly


#ef.add_transaction_costs(selected_prices.iloc[-1], 0.01)  # 1% transaction cost
    # attempts to include transaction costs
weights = ef.efficient_return(target_return=((SP_returns + TSX_returns)/2).mean()) # find optimal weights to achieve market return
weights

OrderedDict([('PEP', 0.056958723391344),
             ('BMY', 0.0457520556468836),
             ('PFE', 0.0181608202641305),
             ('UNH', 0.0477700953144293),
             ('KO', -2e-16),
             ('CL', 0.1004931927272332),
             ('MRK', -4e-16),
             ('LMT', 0.0711330168488515),
             ('PG', 0.1696320368008865),
             ('ABBV', -1e-16),
             ('T.TO', 0.1589734710424723),
             ('LLY', 0.0420448099624986),
             ('PM', 0.0),
             ('ABT', 0.0012940010677272),
             ('TD.TO', 0.1166852462225282),
             ('RY.TO', 0.0629548377238307),
             ('MO', -1e-16),
             ('BIIB', 0.1081476929871851)])

In [138]:
# Step 4: Allocate funds
ranked_stocks = {'LLY': 241.08,   # "stocks"
                 'BLK': 211.54,
                 'AXP': 210.5,
                 'RY.TO': 197.22,
                 'BK': 196.75,
                 'CL': 193.52,
                 'CAT': 193.34,
                 'BAC': 191.68,
                 'TXN': 191.35,
                 'C': 188.35,
                 'AMZN': 187.5,
                 'PG': 184.23,
                 'ABBV': 180.09,
                 'KO': 180.04,
                 'AIG': 176.56,
                 'QCOM': 174.33,
                 'USB': 168.84,
                 'MO': 167.77,
                 'PM': 160.04,
                 'LMT': 158.41,
                 'PYPL': 154.33,
                 'UNP': 148.9,
                 'PEP': 142.54,
                 'MRK': 138.51}

stockslist = list(ranked_stocks.keys())
weightlist = [0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
 0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.14999999,
 0.02083333, 0.02083333, 0.02083334, 0.02083333, 0.02083333, 0.02083333,
 0.02083333, 0.02083333, 0.02083333, 0.13333333, 0.15, 0.15]             # "best_weight"
weights = dict(zip(stockslist, weightlist))
total_money = 1000000
final_allocation = {}
for stock, weight in weights.items():
    if weight > 0:
        allocation = total_money * weight  
        ticker = yf.Ticker(stock)
        price = ticker.fast_info["lastPrice"]  # need to make sure "end_date" is the most recent trading day
        '''
        currency = ticker.fast_info["currency"]     
        if currency == 'USD':
            exchange = 'USDCAD=X'
            exchange_ticker = yf.Ticker(exchange)
            exchange_rate = exchange_ticker.fast_info["lastPrice"]
            price *= exchange_rate
        elif currency == 'CAD':
            continue
        '''
        num_shares = (allocation / price)
        # considering transaction costs
        if num_shares > 3950:
            num_shares = (allocation - 3.95)/price
            transaction_cost = 3.95
        else:
            num_shares = allocation/(price+0.001)
            transaction_cost = num_shares*0.001
        total_cost = num_shares * price
        #transaction_cost = total_cost * 0.01
        final_allocation[stock] = {
            "Number of Shares": num_shares,
            "Transaction Cost": transaction_cost,
            "Total Money Spent": total_cost + transaction_cost,
            "Cumulative Transaction Cost": np.round(sum(stock["Transaction Cost"] for stock in final_allocation.values()), 2),
            "Cumulative Total": np.round(total_cost + transaction_cost+ sum(stock["Total Money Spent"] for stock in final_allocation.values()), 2)
        }
final = pd.DataFrame(final_allocation)

print(f"Total Money Spent: ${sum(a['Total Money Spent'] for a in final_allocation.values()):.2f}")
print(f"Total Cost: ${sum(a['Transaction Cost'] for a in final_allocation.values()):.2f}")
display(final)

Total Money Spent: $999999.93
Total Cost: $7.68


,LLY,BLK,AXP,RY.TO,BK,CL,CAT,BAC,TXN,C,...,AIG,QCOM,USB,MO,PM,LMT,PYPL,UNP,PEP,MRK
Number of Shares,27.780700,20.265866,71.103273,119.210411,264.110885,221.040944,53.474875,448.404692,105.112135,302.146900,...,274.191430,134.010003,405.388692,371.817930,158.777305,38.437097,245.615238,557.831017,935.506224,1502.087893
Transaction Cost,0.027781,0.020266,0.071103,0.119210,0.264111,0.221041,0.053475,0.448405,0.105112,0.302147,...,0.274191,0.134010,0.405389,0.371818,0.158777,0.038437,0.245615,0.557831,0.935506,1.502088
Total Money Spent,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,...,20833.340000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,20833.330000,133333.330000,150000.000000,150000.000000
Cumulative Transaction Cost,0.000000,0.030000,0.050000,0.120000,0.240000,0.500000,0.720000,0.780000,1.230000,1.330000,...,3.050000,3.330000,3.460000,3.870000,4.240000,4.400000,4.440000,4.680000,5.240000,6.180000
Cumulative Total,20833.330000,41666.660000,62499.990000,83333.320000,104166.650000,124999.980000,145833.310000,166666.640000,187499.970000,208333.300000,...,441666.620000,462499.950000,483333.280000,504166.610000,524999.940000,545833.270000,566666.600000,699999.930000,849999.930000,999999.930000


In [142]:
portfolio =0
cost = 7.68

for column, values in final.items():
    num = values.iloc[0]
    #print(num)
    #price = values.iloc[1]
    cur_price = yf.Ticker(column).fast_info['lastPrice']
    portfolio += cur_price * num

rate = (portfolio-1000000)/(1000000-cost)
print(portfolio, rate)

999992.2523133061 -7.7477461966013e-06
